<a href="https://colab.research.google.com/github/SeongGwan-a/Project-Classification-of-novelists/blob/main/DNN/DNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 라이브러리 불러오기
import numpy as np; import pandas as pd
from collections import defaultdict

import torch
import keras
import keras.backend as K
from keras.layers import Dense, GlobalAveragePooling1D, Embedding, BatchNormalization
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical

from sklearn.model_selection import train_test_split
from nltk import word_tokenize
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# 데이터 불러오기
url_train      = 'https://raw.githubusercontent.com/moch1996/firststep/master/train.csv'
url_test       = 'https://raw.githubusercontent.com/moch1996/firststep/master/test_x.csv'
url_submission = 'https://raw.githubusercontent.com/moch1996/firststep/master/sample_submission.csv'


train             = pd.read_csv(url_train)
test              = pd.read_csv(url_test)
sample_submission = pd.read_csv(url_submission)

In [ ]:
# train test 분리
X_train = np.array([x for x in train['text']])
X_test = np.array([x for x in test['text']])
y_train = np.array([x for x in train['author']])
y = to_categorical(y_train)
y

array([[0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0.],
       ...,
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0.]], dtype=float32)

In [ ]:
?set

In [ ]:
# Data 전처리
def preprocess(text):
    text = text.replace("' ", " ' ")
    signs = set(',.:;"?!')
    prods = set(text) & signs
    if not prods:
        return text

    for sign in prods:
        text = text.replace(sign, ' {} '.format(sign) )
    return text

In [ ]:
#Train 데이터로부터 n-gram language 모델 사용
def create_docs(df, n_gram_max=3):
    def add_ngram(q, n_gram_max):
            ngrams = []
            for n in range(2, 4):
                for w_index in range(len(q)-n+1):
                    ngrams.append('--'.join(q[w_index:w_index+n]))
            return q + ngrams
        
    docs = []
    for doc in df.text:
        doc = preprocess(doc).split()
        docs.append(' '.join(add_ngram(doc, n_gram_max)))
    
    return docs

In [ ]:
min_count = 2

docs = create_docs(train)
tokenizer = Tokenizer(lower=False, filters='')
tokenizer.fit_on_texts(docs) 
num_words = sum([1 for _, v in tokenizer.word_counts.items() if v >= min_count]) #2번이상 사용한 단어로 토큰화

tokenizer = Tokenizer(num_words=num_words, lower=False, filters='')
tokenizer.fit_on_texts(docs) #빈도수를 기준으로 단어 집합을 생성 
docs = tokenizer.texts_to_sequences(docs)

maxlen = 256

docs = pad_sequences(sequences=docs, maxlen=maxlen)  #Padding

In [ ]:
# DNN architecture
input_dim = np.max(docs) + 1
embedding_dims = 20

def create_model(embedding_dims=20, optimizer='adam'):
    model = Sequential()
    model.add(Embedding(input_dim=input_dim, output_dim=embedding_dims))
    model.add(GlobalAveragePooling1D())
    model.add(Dense(24, activation='relu'))
    model.add(Dense(5, activation='softmax'))

    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])
    return model

In [ ]:
epochs = 25
train_x, test_x, train_y, test_y = train_test_split(docs, y, test_size=0.2, random_state=7)

In [ ]:
import time
start = time.time()
model = create_model()
hist = model.fit(train_x, train_y,
                 batch_size=16,
                 validation_data=(test_x, test_y),
                 epochs=epochs,
                 callbacks=[EarlyStopping(patience=2, monitor='val_loss')])

print("실행시간 : ", time.time()-start,'s')

Epoch 1/25
2744/2744 [==============================] - 237s 86ms/step - loss: 1.1695 - accuracy: 0.5368 - val_loss: 0.7928 - val_accuracy: 0.7167
Epoch 2/25
2744/2744 [==============================] - 237s 86ms/step - loss: 0.5020 - accuracy: 0.8323 - val_loss: 0.5633 - val_accuracy: 0.8024
Epoch 3/25
2744/2744 [==============================] - 237s 86ms/step - loss: 0.2481 - accuracy: 0.9263 - val_loss: 0.5065 - val_accuracy: 0.8202
Epoch 4/25
2744/2744 [==============================] - 238s 87ms/step - loss: 0.1295 - accuracy: 0.9652 - val_loss: 0.5008 - val_accuracy: 0.8261
Epoch 5/25
2744/2744 [==============================] - 237s 86ms/step - loss: 0.0702 - accuracy: 0.9822 - val_loss: 0.5591 - val_accuracy: 0.8181
Epoch 6/25
2744/2744 [==============================] - 236s 86ms/step - loss: 0.0400 - accuracy: 0.9902 - val_loss: 0.5623 - val_accuracy: 0.8286
실행시간 :  1422.9051268100739 s


In [ ]:
def test(t=test) :
  docs = create_docs(t)
  token = tokenizer.texts_to_sequences(docs)
  padding = pad_sequences(sequences=token, maxlen=maxlen)
  pred = model.predict(padding)
  return pred

In [ ]:
start = time.time()
result = test()
print("실행시간 : ", time.time()-start,'s')

실행시간 :  7.818409442901611 s


In [ ]:
result

array([[4.7602825e-06, 9.9753654e-01, 2.4538797e-03, 4.8136581e-06,
        1.3662501e-09],
       [4.9125015e-06, 9.9936026e-01, 6.1134802e-04, 2.2133459e-05,
        1.3951578e-06],
       [1.0000000e+00, 4.0021320e-08, 7.6073777e-13, 1.1941888e-08,
        6.4450658e-09],
       ...,
       [4.0212985e-06, 9.9999595e-01, 3.9731868e-10, 5.8444437e-11,
        1.2672596e-10],
       [8.9800416e-09, 9.9999988e-01, 6.7974490e-08, 3.0172468e-12,
        1.8840457e-10],
       [9.9962640e-01, 2.0243125e-09, 6.8953415e-10, 3.1813968e-11,
        3.7356245e-04]], dtype=float32)

In [ ]:
sample_submission.columns

Index(['index', '0', '1', '2', '3', '4'], dtype='object')

In [ ]:
sample_submission[sample_submission.columns[1:6]] = result

In [ ]:
sample_submission[:10]

,index,0,1,2,3,4
0,0,0.000240,9.978201e-01,1.876129e-03,6.338787e-05,3.951650e-07
1,1,0.000145,9.994617e-01,1.743591e-05,3.000248e-04,7.591779e-05
2,2,0.999995,4.310918e-06,8.408298e-09,2.967679e-07,2.023247e-07
3,3,0.002097,2.319220e-08,9.977470e-01,1.025528e-05,1.459676e-04
4,4,0.999756,2.167622e-04,6.107567e-07,8.967370e-06,1.798590e-05
5,5,0.000078,1.720144e-05,2.455003e-04,9.996545e-01,5.063512e-06
6,6,0.008161,1.968044e-04,1.750527e-01,8.157897e-01,8.000879e-04
7,7,0.991946,4.740449e-03,1.171765e-04,2.021521e-04,2.994342e-03
8,8,0.999992,7.491814e-06,5.744236e-09,1.526828e-13,5.503095e-08
9,9,0.358187,6.203240e-01,4.795769e-03,1.516228e-02,1.531404e-03


In [ ]:
from google.colab import files
sample_submission.to_csv('DNN.csv')
files.download('DNN.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>